# Подготовка к работе с данными

Импортируем нужные библиотеки

In [ ]:
!pip install -q datasets transformers evaluate

In [ ]:
from scipy import stats
import statistics
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import unicodedata
import pandas as pd
import numpy as np
from datasets import Dataset

Импортируем исходный датасет blinoff/kinopoisk

In [ ]:
df = pd.read_json("hf://datasets/blinoff/kinopoisk/kinopoisk.jsonl", lines=True)

In [ ]:
df

# Выбор предложений для разметки

Средний объем тренировочного датасета из соревнования RuArg-2022 -- 2296. Помимо тренировочной выборки, которая стандартно составляет 70% от общего датасета, также нужны валидационная и тестовая, включающих в себя по 15% от общего количества данных.
Таким образом, нам нужно около 3280 текстов (2296 - 492 - 492).
В целях экономии ресурсов, выберем самые короткие. В датасете blinoff/kinopoisk подходящая длина отзыва -- 93 слова или меньше (суммарно 3242 отзыва).
Отфильтруем нужные строки и проанализируем получившийся набор данных.

In [ ]:
short_texts = []

for index, row in df.iterrows():
  row["content"] = row["content"].replace('\n', '')
  row["content"] = unicodedata.normalize("NFKD", row["content"])
  if len(row["content"].split()) <= 93:
    short_texts.append(row)

In [ ]:
short_texts = pd.DataFrame(short_texts)

print(len(short_texts))

Рассмотрим количество отзывов в каждом классе (Good, Neutral, Bad)

In [ ]:
grade_counts = short_texts['grade3'].value_counts()
grade_counts

In [ ]:
print(2199/3242)
print(724/3242)
print(319/3242)

Как видно, классы не сбалансированы, что может повлиять на итоговые результаты.
Рассмотрим отзывы длиной до 156 слов и выберем из них около 3280 с нужным соотношением (порядка 1100 отзывов каждого типа).

In [ ]:
short_texts = []

for index, row in df.iterrows():
  row["content"] = row["content"].replace('\n', '') #в этой и следующей строчке -- предобработка текстов
  row["content"] = unicodedata.normalize("NFKD", row["content"])
  if len(row["content"].split()) <= 156:
    short_texts.append(row)

short_texts = pd.DataFrame(short_texts)
print(len(short_texts))

grade_counts = short_texts['grade3'].value_counts()
grade_counts

Количество отрицательных отзывов - 1084, в двух других классах значительно больше, поэтому выберём 1098 нейтральных и 1098 положительных отзывов случайным образом

In [ ]:
short_bad = short_texts[short_texts["grade3"] == "Bad"]

short_neut = short_texts[short_texts["grade3"] == "Neutral"].sample(1098)

short_good = short_texts[short_texts["grade3"] == "Good"].sample(1098)

Объединяем 3 класса отзывов в единый датасет

In [ ]:
result = pd.concat([short_bad, short_neut, short_good], ignore_index=True)

In [ ]:
print(len(result))
grade_counts = result['grade3'].value_counts()
grade_counts

Как видно, отзывы сгруппированы по фильмам, поэтому перемешаем их случайным образом

In [ ]:
result

In [ ]:
result = result.sample(frac=1)

In [ ]:
result

Экспортируем данные в формат Excel для дальнейшей ручной разметки

In [ ]:
result.to_excel("blinoff.xlsx")

# Создание итогового размеченного датасета

Загружаем размеченные данные

In [ ]:
dataframe = pd.read_csv("processed_data_final.csv", encoding="utf8")

In [ ]:
dataframe

Старые отзывы были автоматически отнесены к нейтральным при переходе на новую троичную систему. Также в отдельных случаях содержание отзывов и их категория не совпадала. Подобные несоответствия помечались разметчиками в столбце NEW_grade3.

In [ ]:
# заполним NEW_grade3 для остальных отзывов

dataframe['NEW_grade3'] = dataframe['NEW_grade3'].fillna(0)
dataframe["NEW_grade3"] = np.where(dataframe["NEW_grade3"] == 0, dataframe["grade3"], dataframe["NEW_grade3"]) #берём оценку из исходного столбца
print(dataframe["NEW_grade3"].head(10))

Определяем класс для аргументов на основе размеченного количества и в соответствии со следующими правилами:
1.   В каждом отзыве подсчитывалось количество аргументов «за» и количество аргументов «против»
2.   На основании полученных чисел выставлялся лейбл:

*   если в обеих колонках «0» — Neutral
*   если в одной колонке «0» — лейбл, соответсвующий колонке с ненулевым значением
*   если в обеих колонках ненулевые значения и разность двух чисел меньше 3 — Neutral
*   если в обеих колонках ненулевые значения и разность чисел не меньше 3 — лейбл, соответствующий колонке с наибольшим значением

In [ ]:
# добавляем столбец из нулей

dataframe.insert(13, "arg_label", 0)

In [ ]:
# определяем классы для аргументации

# dataframe["arg_label"] = np.where((dataframe["for"] == 0 and dataframe["against"] == 0), "Neutral", 0)

arg_label = []
for index, row in dataframe.iterrows():
  if row["for"] == 0 and row["against"] == 0:
    arg_label.append("Neutral")
  elif row["for"] == 0:
    arg_label.append("Bad")
  elif row["against"] == 0:
    arg_label.append("Good")
  elif abs(row["for"] - row["against"]) > 2:
    if row["for"] > row["against"]:
      arg_label.append("Good")
    elif row["for"] < row["against"]:
      arg_label.append("Bad")
  elif abs(row["for"] - row["against"]) <= 2:
    arg_label.append("Neutral")
  else:
    arg_label.append(0)
    print(row)

dataframe["arg_label"] = arg_label

In [ ]:
dataframe

In [ ]:
# добавляем столбец "век"
dataframe.insert(14, "years", 0)

years = []
for index, row in dataframe.iterrows():
  # years.append(int(row["movie_name"].split(" ")[-1][1:-1]))
  year = int(row["movie_name"].split(" ")[-1][1:-1])
  if year <= 2000:
    years.append(20)
  else:
    years.append(21)
dataframe["years"] = years

In [ ]:
count = dataframe["years"].value_counts()
print(count)

In [ ]:
dataframe

In [ ]:
# добавляем столбец с рейтингом
dataframe.insert(15, "half", 0)
part = []
for index, row in dataframe.iterrows():
  part.append(int(row["part"][-3:]))

dataframe["half"] = part

In [ ]:
count = dataframe["half"].value_counts()
print(count)

In [ ]:
dataframe

In [ ]:
# добавляем столбец с соответствием
dataframe.insert(16, "congruence", 0)
are_congruent = []
for index, row in dataframe.iterrows():
  are_congruent.append(int(row["NEW_grade3"] == row["arg_label"]))

dataframe["congruence"] = are_congruent

In [ ]:
count = dataframe["congruence"].value_counts()
print(count)

In [ ]:
dataframe

Наш датасет можно увидеть по этой ссылке: https://huggingface.co/datasets/otipl2125/film_review_argumentation

# Подготовка обучающей, валидационной и тестовой выборок

Разделяем датасет на 3 выборки

In [ ]:
train = dataframe[0:2295]
val = dataframe[2296:2788]
test = dataframe[2788:]

print(len(train), len(val), len(test))

Переводим метки из текстового формата в числовой

In [ ]:
label_dict = {"Bad": 0, "Neutral": 1, "Good": 2}
train["raw_NEW_grade3"] = train['NEW_grade3']
train['NEW_grade3'] = train['raw_NEW_grade3'].map(label_dict)
train['raw_arg_label'] = train['arg_label']
train['arg_label'] = train["raw_arg_label"].map(label_dict)
val["raw_NEW_grade3"] = val['NEW_grade3']
val['NEW_grade3'] = val['raw_NEW_grade3'].map(label_dict)
val['raw_arg_label'] = val['arg_label']
val['arg_label'] = val["raw_arg_label"].map(label_dict)
test["raw_NEW_grade3"] = test['NEW_grade3']
test['NEW_grade3'] = test['raw_NEW_grade3'].map(label_dict)
test['raw_arg_label'] = test['arg_label']
test['arg_label'] = test["raw_arg_label"].map(label_dict)
train.head()

In [ ]:
train = Dataset.from_pandas(train, preserve_index=False)
print(type(train))
val = Dataset.from_pandas(val, preserve_index=False)
test = Dataset.from_pandas(test, preserve_index=False)

In [ ]:
from datasets import DatasetDict
dataset_dict = DatasetDict({"train": train,
                            "validation": val,
                            "test": test})
dataset_dict

# Работа с моделями

Так как мы планируем использовать метод дообучения энкодерной модели и работать с русскоязычными текстами, воспользуемся моделью DeepPavlov/rubert-base-cased, с помощью которой было создано базовое решение организаторов RuArg-2022, а затем моделью ai-forever/ru-Roberta-large, учитывая опыт исследований, связанных с IMDb.

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
DEVICE = "cuda"
#DEVICE = "cpu"

In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

## DeepPavlov/rubert-base-cased

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Токенизируем данные.

In [ ]:
def tokenize_function(example):
    return tokenizer(example["content"])

In [ ]:
tokenized_dataset = dataset_dict.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['Unnamed: 0', 'part', 'movie_name', "author", "review_id", "date", "title", "grade10", "grade3", "content", "for", "against", "raw_NEW_grade3", "raw_arg_label"])
tokenized_dataset

Создадим объекты класса DataLoader для деления на батчи и паддинга.

In [ ]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(
    tokenized_dataset["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)

val_dataloader = DataLoader(
    tokenized_dataset["validation"], batch_size=8, collate_fn=data_collator
)

test_dataloader = DataLoader(
    tokenized_dataset["test"], batch_size=8, collate_fn=data_collator
)

Для дообучения модели для задачи классификации возьмем предобученную модель с незамороженными весами и добавим два линейных слоя: для определения позиции и для классификации доводов.

In [ ]:
from torch import nn

class CustomBertForSequenceClassification(nn.Module):

  def __init__(self, n_labels):
    super().__init__()
    self.bert = AutoModel.from_pretrained(checkpoint)
    self.drop = nn.Dropout(p=0.3)
    self.stance_out = nn.Linear(self.bert.config.hidden_size, n_labels)
    self.argument_out = nn.Linear(self.bert.config.hidden_size, n_labels)

  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      return_dict=False)
    output = self.drop(pooled_output)
    stance_logits = self.stance_out(output)
    argument_logits = self.argument_out(output)

    return {"stance": stance_logits, "argument": argument_logits}

Определим функции для обучения и валидации

In [ ]:
import numpy as np

def train_epoch(current_class, model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
  model = model.train() # переводим модель в состояние обучения

  losses = [] # значения функции потерь
  # значения accuracy
  stance_correct_predictions = 0
  argument_correct_predictions = 0

  for d in data_loader: # итерация по батчам
    input_ids = d["input_ids"].to(device) # индексы токенов
    attention_mask = d["attention_mask"].to(device) # маски внимания
    # метки классов
    stance_targets = d[f"NEW_grade3"].to(device)
    argument_targets = d[f"arg_label"].to(device)

    # применяем модель
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # позиция максимального значения
    stance_preds = torch.argmax(outputs["stance"], dim=1)
    argument_preds = torch.argmax(outputs["argument"], dim=1)
    # подсчет функции потерь
    stance_loss = loss_fn(outputs["stance"], stance_targets)
    argument_loss = loss_fn(outputs["argument"], argument_targets)
    loss = stance_loss + argument_loss

    # количество совпадений
    stance_correct_predictions += torch.sum(stance_preds == stance_targets)
    argument_correct_predictions += torch.sum(argument_preds == argument_targets)
    losses.append(loss.item())

    loss.backward() # подсчет градиента
    optimizer.step() # обновление весов
    scheduler.step() # изменение скорости обучения
    optimizer.zero_grad() # обнуление градиентов

  return stance_correct_predictions / n_examples, argument_correct_predictions / n_examples, np.mean(losses) # accuracy, среднее значение ошибки

In [ ]:
def eval_model(current_class, model, data_loader, loss_fn, device, n_examples):
  model = model.eval() # переводим модель в состояние валидации

  losses = [] # значения функции потерь
  # значения accuracy
  stance_correct_predictions = 0
  argument_correct_predictions = 0

  with torch.no_grad(): # градиент не считается
    for d in data_loader: # итерация по батчам
      input_ids = d["input_ids"].to(device) # индексы токенов
      attention_mask = d["attention_mask"].to(device) # маски внимания
      # метки классов
      stance_targets = d[f"NEW_grade3"].to(device)
      argument_targets = d[f"arg_label"].to(device)

      # применяем модель
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      # позиция максимального значения
      stance_preds = torch.argmax(outputs["stance"], dim=1)
      argument_preds = torch.argmax(outputs["argument"], dim=1)
      # подсчет функции потерь
      stance_loss = loss_fn(outputs["stance"], stance_targets)
      argument_loss = loss_fn(outputs["argument"], argument_targets)
      loss = stance_loss + argument_loss

      # количество совпадений
      stance_correct_predictions += torch.sum(stance_preds == stance_targets)
      argument_correct_predictions += torch.sum(argument_preds == argument_targets)
      losses.append(loss.item())

  return stance_correct_predictions / n_examples, argument_correct_predictions / n_examples, np.mean(losses) # accuracy, среднее значение ошибки

In [ ]:
from transformers import AutoModel, get_linear_schedule_with_warmup
from torch.optim import AdamW

def fine_tuning(current_class, epochs):

  print(f"Training model:\n")

  # Загрузка предобученной модели
  bert_model = AutoModel.from_pretrained(checkpoint)
  # Добавление линейных слоев
  model = CustomBertForSequenceClassification(n_labels = 4).to(device)

  EPOCHS = epochs
  # Обучение всех слоев
  optimizer = AdamW(model.parameters(), lr=2e-5)
  total_steps = len(train_dataloader) * EPOCHS
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

  loss_fn = nn.CrossEntropyLoss().to(device)

  for epoch in range(EPOCHS): # итерация по эпохам
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    # обучение
    train_stance_acc, train_argument_acc, train_loss = train_epoch(current_class, model, train_dataloader, loss_fn, optimizer, device, scheduler, len(tokenized_dataset["train"]))

    print(f'Train loss {train_loss} stance accuracy {train_stance_acc} argument accuracy {train_argument_acc}')

    # валидация
    val_stance_acc, val_argument_acc, val_loss = eval_model(current_class, model, val_dataloader, loss_fn, device, len(tokenized_dataset["test"]))

    print(f'Val loss {val_loss} stance accuracy {val_stance_acc} argument accuracy {val_argument_acc}')
    print()

  return bert_model, model

Обучаем модель.

In [ ]:
bert_model, supervised_model = fine_tuning("", epochs = 2)

Напишем функции для получения предсказаний обученной модели и подсчета макро F1-меры.

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()

  # предсказанные метки
  stance_predictions = []
  argument_predictions = []

  with torch.no_grad(): # градиент не считается
    for d in data_loader: # итерация по батчам
      input_ids = d["input_ids"].to(device) # индексы токенов
      attention_mask = d["attention_mask"].to(device) # маски внимания

      # применяем модель
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      # позиция максимального значения
      stance_preds = torch.argmax(outputs["stance"], dim=1)
      argument_preds = torch.argmax(outputs["argument"], dim=1)

      stance_predictions.extend(stance_preds)
      argument_predictions.extend(argument_preds)

  stance_predictions = torch.stack(stance_predictions).cpu()
  argument_predictions = torch.stack(argument_predictions).cpu()


  stance_predictions = [x.item() for x in stance_predictions]
  argument_predictions = [x.item() for x in argument_predictions]

  return stance_predictions, argument_predictions

In [ ]:
import evaluate

def compute_metrics(preds, labels):
    metric = evaluate.load("f1")
    return metric.compute(predictions=preds, references=labels, average="macro")

Посчитаем метрику для модели.

In [ ]:
def validation_score(model):
  val_stance_predictions, val_argument_predictions = get_predictions(model, val_dataloader)
  tokenized_dataset["validation"] = tokenized_dataset["validation"].add_column(f"stance_predicdions", val_stance_predictions)
  tokenized_dataset["validation"] = tokenized_dataset["validation"].add_column(f"argument_predicdions", val_argument_predictions)
  # filtered_validation = tokenized_dataset["validation"].filter(lambda example: example[f"NEW_grade3"]!=-1)
  stance_f1 = compute_metrics(tokenized_dataset["validation"][f"stance_predicdions"], tokenized_dataset["validation"][f"NEW_grade3"])
  argument_f1 = compute_metrics(tokenized_dataset["validation"][f"argument_predicdions"], tokenized_dataset["validation"][f"arg_label"])
  return stance_f1['f1'], argument_f1['f1']

In [ ]:
stance_f1, argument_f1 = validation_score(supervised_model)
print(f"Stance F1 = {stance_f1}")
print(f"Argument F1 = {argument_f1}")

Посчитаем метрику для модели для фильмов XX и XXI века.

In [ ]:
def validation_score_years(model):


    val_dataset_year_20 = tokenized_dataset["validation"].filter(lambda example: example["years"] == 20)
    val_dataset_year_21 = tokenized_dataset["validation"].filter(lambda example: example["years"] == 21)

    val_dataloader_for_year_20 = DataLoader(
    val_dataset_year_20, shuffle=False, batch_size=8, collate_fn=data_collator)

    val_dataloader_for_year_21 = DataLoader(
    val_dataset_year_21, shuffle=False, batch_size=8, collate_fn=data_collator)

    val_stance_predictions_20, val_argument_predictions_20 = get_predictions(model, val_dataloader_for_year_20)
    val_dataset_year_20 = val_dataset_year_20.add_column("stance_predictions", val_stance_predictions_20)
    val_dataset_year_20 = val_dataset_year_20.add_column("argument_predictions", val_argument_predictions_20)


    stance_f1_20 = compute_metrics(val_dataset_year_20["stance_predictions"], val_dataset_year_20["NEW_grade3"])
    argument_f1_20 = compute_metrics(val_dataset_year_20["argument_predictions"], val_dataset_year_20["arg_label"])


    val_stance_predictions_21, val_argument_predictions_21 = get_predictions(model, val_dataloader_for_year_21)
    val_dataset_year_21 = val_dataset_year_21.add_column("stance_predictions", val_stance_predictions_21)
    val_dataset_year_21 = val_dataset_year_21.add_column("argument_predictions", val_argument_predictions_21)


    stance_f1_21 = compute_metrics(val_dataset_year_21["stance_predictions"], val_dataset_year_21["NEW_grade3"])
    argument_f1_21 = compute_metrics(val_dataset_year_21["argument_predictions"], val_dataset_year_21["arg_label"])

    return (stance_f1_20['f1'], argument_f1_20['f1']), (stance_f1_21['f1'], argument_f1_21['f1'])

In [ ]:
f1_20, f1_21 = validation_score_years(supervised_model)
print(f"Century 20 = {f1_20}")
print(f"Century 21 = {f1_21}")

Посчитаем метрику для модели для фильмов из вершины рейтинга (top-250) и низа рейтинга (bottom-100)

In [ ]:
def validation_score_parts(model):


    val_dataset_part_250 = tokenized_dataset["validation"].filter(lambda example: example["half"] == 250)
    val_dataset_part_100 = tokenized_dataset["validation"].filter(lambda example: example["half"] == 100)

    val_dataloader_for_part_250 = DataLoader(
    val_dataset_part_250, shuffle=False, batch_size=8, collate_fn=data_collator)

    val_dataloader_for_part_100 = DataLoader(
    val_dataset_part_100, shuffle=False, batch_size=8, collate_fn=data_collator)

    val_stance_predictions_part_250, val_argument_predictions_part_250 = get_predictions(model, val_dataloader_for_part_250)
    val_dataset_part_250 = val_dataset_part_250.add_column("stance_predictions", val_stance_predictions_part_250)
    val_dataset_part_250 = val_dataset_part_250.add_column("argument_predictions", val_argument_predictions_part_250)


    stance_f1_part_250 = compute_metrics(val_dataset_part_250["stance_predictions"], val_dataset_part_250["NEW_grade3"])
    argument_f1_part_250 = compute_metrics(val_dataset_part_250["argument_predictions"], val_dataset_part_250["arg_label"])


    val_stance_predictions_part_100, val_argument_predictions_part_100 = get_predictions(model, val_dataloader_for_part_100)
    val_dataset_part_100 = val_dataset_part_100.add_column("stance_predictions", val_stance_predictions_part_100)
    val_dataset_part_100 = val_dataset_part_100.add_column("argument_predictions", val_argument_predictions_part_100)


    stance_f1_part_100 = compute_metrics(val_dataset_part_100["stance_predictions"], val_dataset_part_100["NEW_grade3"])
    argument_f1_part_100 = compute_metrics(val_dataset_part_100["argument_predictions"], val_dataset_part_100["arg_label"])

    return (stance_f1_part_250['f1'], argument_f1_part_250['f1']), (stance_f1_part_100['f1'], argument_f1_part_100['f1'])

In [ ]:
part_250_f1, part_100_f1 = validation_score_parts(supervised_model)
print(f"Part 250 = {part_250_f1}")
print(f"Part 100 = {part_100_f1}")

Посчитаем метрику для модели для конгруэнтных и неконгруэнтных меток

In [ ]:
def validation_score_congruence(model):


    val_dataset_congruent = tokenized_dataset["validation"].filter(lambda example: example["congruence"] == 1)
    val_dataset_not_congruent = tokenized_dataset["validation"].filter(lambda example: example["congruence"] == 0)

    val_dataloader_for_congruent = DataLoader(
    val_dataset_congruent, shuffle=False, batch_size=8, collate_fn=data_collator)

    val_dataloader_for_not_congruent = DataLoader(
    val_dataset_not_congruent, shuffle=False, batch_size=8, collate_fn=data_collator)

    val_stance_predictions_congruent, val_argument_predictions_congruent = get_predictions(model, val_dataloader_for_congruent)
    val_dataset_congruent = val_dataset_congruent.add_column("stance_predictions", val_stance_predictions_congruent)
    val_dataset_congruent = val_dataset_congruent.add_column("argument_predictions", val_argument_predictions_congruent)


    stance_f1_congruent = compute_metrics(val_dataset_congruent["stance_predictions"], val_dataset_congruent["NEW_grade3"])
    argument_f1_congruent = compute_metrics(val_dataset_congruent["argument_predictions"], val_dataset_congruent["arg_label"])


    val_stance_predictions_not_congruent, val_argument_predictions_not_congruent = get_predictions(model, val_dataloader_for_not_congruent)
    val_dataset_not_congruent = val_dataset_not_congruent.add_column("stance_predictions", val_stance_predictions_not_congruent)
    val_dataset_not_congruent = val_dataset_not_congruent.add_column("argument_predictions", val_argument_predictions_not_congruent)


    stance_f1_not_congruent = compute_metrics(val_dataset_not_congruent["stance_predictions"], val_dataset_not_congruent["NEW_grade3"])
    argument_f1_not_congruent = compute_metrics(val_dataset_not_congruent["argument_predictions"], val_dataset_not_congruent["arg_label"])

    return (stance_f1_congruent['f1'], argument_f1_congruent['f1']), (stance_f1_not_congruent['f1'], argument_f1_not_congruent['f1'])

In [ ]:
congruent_f1, not_congruent_f1 = validation_score_congruence(supervised_model)
print(f"Congruent = {congruent_f1}")
print(f"Not congruent = {not_congruent_f1}")

## ai-forever/ru-Roberta-large

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "ai-forever/ruRoberta-large"
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruRoberta-large")

Токенизируем данные

In [ ]:
tokenized_dataset = dataset_dict.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['Unnamed: 0', 'part', 'movie_name', "author", "review_id", "date", "title", "grade10", "grade3", "content", "for", "against", "raw_NEW_grade3", "raw_arg_label"])
tokenized_dataset

Создадим объекты класса DataLoader для деления на батчи и паддинга.

In [ ]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(
    tokenized_dataset["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)

val_dataloader = DataLoader(
    tokenized_dataset["validation"], batch_size=8, collate_fn=data_collator
)

test_dataloader = DataLoader(
    tokenized_dataset["test"], batch_size=8, collate_fn=data_collator
)

Обучаем модель

In [ ]:
roberta_model, supervised_model = fine_tuning("", epochs = 2)

Посчитаем метрику для модели.

In [ ]:
def validation_score(model):
  val_stance_predictions, val_argument_predictions = get_predictions(model, val_dataloader)
  tokenized_dataset["validation"] = tokenized_dataset["validation"].add_column(f"stance_predictions", val_stance_predictions)
  tokenized_dataset["validation"] = tokenized_dataset["validation"].add_column(f"argument_predictions", val_argument_predictions)
  # filtered_validation = tokenized_dataset["validation"].filter(lambda example: example[f"NEW_grade3"]!=-1)
  stance_f1 = compute_metrics(tokenized_dataset["validation"][f"stance_predictions"], tokenized_dataset["validation"][f"NEW_grade3"])
  argument_f1 = compute_metrics(tokenized_dataset["validation"][f"argument_predictions"], tokenized_dataset["validation"][f"arg_label"])
  return stance_f1['f1'], argument_f1['f1']

In [ ]:
stance_f1, argument_f1 = validation_score(supervised_model)
print(f"Stance F1 = {stance_f1}")
print(f"Argument F1 = {argument_f1}")

Посчитаем метрику для модели для фильмов XX и XXI вв.

In [ ]:
def validation_score_years(model):


    val_dataset_year_20 = tokenized_dataset["validation"].filter(lambda example: example["years"] == 20)
    val_dataset_year_21 = tokenized_dataset["validation"].filter(lambda example: example["years"] == 21)

    val_dataloader_for_year_20 = DataLoader(
    val_dataset_year_20, shuffle=False, batch_size=8, collate_fn=data_collator)

    val_dataloader_for_year_21 = DataLoader(
    val_dataset_year_21, shuffle=False, batch_size=8, collate_fn=data_collator)

    val_stance_predictions_20, val_argument_predictions_20 = get_predictions(model, val_dataloader_for_year_20)
    val_dataset_year_20 = val_dataset_year_20.add_column("stance_predictions", val_stance_predictions_20)
    val_dataset_year_20 = val_dataset_year_20.add_column("argument_predictions", val_argument_predictions_20)


    stance_f1_20 = compute_metrics(val_dataset_year_20["stance_predictions"], val_dataset_year_20["NEW_grade3"])
    argument_f1_20 = compute_metrics(val_dataset_year_20["argument_predictions"], val_dataset_year_20["arg_label"])


    val_stance_predictions_21, val_argument_predictions_21 = get_predictions(model, val_dataloader_for_year_21)
    val_dataset_year_21 = val_dataset_year_21.add_column("stance_predictions", val_stance_predictions_21)
    val_dataset_year_21 = val_dataset_year_21.add_column("argument_predictions", val_argument_predictions_21)


    stance_f1_21 = compute_metrics(val_dataset_year_21["stance_predictions"], val_dataset_year_21["NEW_grade3"])
    argument_f1_21 = compute_metrics(val_dataset_year_21["argument_predictions"], val_dataset_year_21["arg_label"])

    return (stance_f1_20['f1'], argument_f1_20['f1']), (stance_f1_21['f1'], argument_f1_21['f1'])

In [ ]:
stance_f1, argument_f1 = validation_score_years(supervised_model)
print(f"Century 20 = {stance_f1}")
print(f"Century 21 = {argument_f1}")

Посчитаем метрику для модели для фильмов из вершины рейтинга (top-250) и низа рейтинга (bottom-100)

In [ ]:
def validation_score_parts(model):


    val_dataset_part_250 = tokenized_dataset["validation"].filter(lambda example: example["half"] == 250)
    val_dataset_part_100 = tokenized_dataset["validation"].filter(lambda example: example["half"] == 100)

    val_dataloader_for_part_250 = DataLoader(
    val_dataset_part_250, shuffle=False, batch_size=8, collate_fn=data_collator)

    val_dataloader_for_part_100 = DataLoader(
    val_dataset_part_100, shuffle=False, batch_size=8, collate_fn=data_collator)

    val_stance_predictions_part_250, val_argument_predictions_part_250 = get_predictions(model, val_dataloader_for_part_250)
    val_dataset_part_250 = val_dataset_part_250.add_column("stance_predictions", val_stance_predictions_part_250)
    val_dataset_part_250 = val_dataset_part_250.add_column("argument_predictions", val_argument_predictions_part_250)


    stance_f1_part_250 = compute_metrics(val_dataset_part_250["stance_predictions"], val_dataset_part_250["NEW_grade3"])
    argument_f1_part_250 = compute_metrics(val_dataset_part_250["argument_predictions"], val_dataset_part_250["arg_label"])


    val_stance_predictions_part_100, val_argument_predictions_part_100 = get_predictions(model, val_dataloader_for_part_100)
    val_dataset_part_100 = val_dataset_part_100.add_column("stance_predictions", val_stance_predictions_part_100)
    val_dataset_part_100 = val_dataset_part_100.add_column("argument_predictions", val_argument_predictions_part_100)


    stance_f1_part_100 = compute_metrics(val_dataset_part_100["stance_predictions"], val_dataset_part_100["NEW_grade3"])
    argument_f1_part_100 = compute_metrics(val_dataset_part_100["argument_predictions"], val_dataset_part_100["arg_label"])

    return (stance_f1_part_250['f1'], argument_f1_part_250['f1']), (stance_f1_part_100['f1'], argument_f1_part_100['f1'])

In [ ]:
stance_f1, argument_f1 = validation_score_parts(supervised_model)
print(f"Part 250 = {stance_f1}")
print(f"Part 100 = {argument_f1}")

Подсчитаем метрику для конгруэнтных и неконгруэнтных меток

In [ ]:
def validation_score_congruence(model):


    val_dataset_congruent = tokenized_dataset["validation"].filter(lambda example: example["congruence"] == 1)
    val_dataset_not_congruent = tokenized_dataset["validation"].filter(lambda example: example["congruence"] == 0)

    val_dataloader_for_congruent = DataLoader(
    val_dataset_congruent, shuffle=False, batch_size=8, collate_fn=data_collator)

    val_dataloader_for_not_congruent = DataLoader(
    val_dataset_not_congruent, shuffle=False, batch_size=8, collate_fn=data_collator)

    val_stance_predictions_congruent, val_argument_predictions_congruent = get_predictions(model, val_dataloader_for_congruent)
    val_dataset_congruent = val_dataset_congruent.add_column("stance_predictions", val_stance_predictions_congruent)
    val_dataset_congruent = val_dataset_congruent.add_column("argument_predictions", val_argument_predictions_congruent)


    stance_f1_congruent = compute_metrics(val_dataset_congruent["stance_predictions"], val_dataset_congruent["NEW_grade3"])
    argument_f1_congruent = compute_metrics(val_dataset_congruent["argument_predictions"], val_dataset_congruent["arg_label"])


    val_stance_predictions_not_congruent, val_argument_predictions_not_congruent = get_predictions(model, val_dataloader_for_not_congruent)
    val_dataset_not_congruent = val_dataset_not_congruent.add_column("stance_predictions", val_stance_predictions_not_congruent)
    val_dataset_not_congruent = val_dataset_not_congruent.add_column("argument_predictions", val_argument_predictions_not_congruent)


    stance_f1_not_congruent = compute_metrics(val_dataset_not_congruent["stance_predictions"], val_dataset_not_congruent["NEW_grade3"])
    argument_f1_not_congruent = compute_metrics(val_dataset_not_congruent["argument_predictions"], val_dataset_not_congruent["arg_label"])

    return (stance_f1_congruent['f1'], argument_f1_congruent['f1']), (stance_f1_not_congruent['f1'], argument_f1_not_congruent['f1'])

In [ ]:
stance_f1, argument_f1 = validation_score_congruence(supervised_model)
print(f"Congruent = {stance_f1}")
print(f"Not congruent = {argument_f1}")